# Arquivos de temas por unidade da federação
Cria os arquivos vetoriais através da intesecção dos divérsos temas com cada unidade da federação e cria uma camada vetorial de cada tema dissolvendo os polígonos para o cálculo da área total do tema na unidade da federação.

In [1]:
uf_analise = 'Pará'

import geopandas as gpd
import pandas as pd
import locale
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, Markdown
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.precision', 4)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
gpkgfile = '../glebas-federais.gpkg'
uf = gpd.read_file(gpkgfile, layer='uf-brasil')
uf_select = uf[uf['nm_uf']==uf_analise]
glebas = gpd.read_file(gpkgfile, layer='glebas-mais-amazonia')
area_estudo = gpd.read_file(gpkgfile, layer='area-estudo')
sr_ponto = gpd.read_file(gpkgfile, layer='sr_incra')
uc=gpd.read_file(gpkgfile, layer='uc')
ti=gpd.read_file(gpkgfile, layer='ti-portarias')
ti = ti.set_crs(4674, allow_override=True)
pa = gpd.read_file(gpkgfile, layer='pa_brasil')
tq = gpd.read_file(gpkgfile, layer='tq_brasil')
sigef = gpd.read_file(gpkgfile, layer='sigef-particular')
floresta=gpd.read_file(gpkgfile, layer='florestas-publicas')
floresta = floresta[floresta['tipo']=='TIPO B']
mun = gpd.read_file(gpkgfile, layer='municipios-area-estudo')
area_uf = round(uf_select['area_km2'],4)

## Criando as glebas por uf

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    gleba_uf_clip = glebas.overlay(uf_select, how='intersection')
    gleba_uf_clip.to_file(gpkgfile, layer=f'gleba-clip-{estado}')

## Dissolvendo as glebas em polígono único

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins',
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    gleba_uf_clip = gpd.read_file(gpkgfile, layer=f'gleba-clip-{estado}')
    gleba_uf_clip = gleba_uf_clip.dissolve()
    gleba_uf_clip.to_file(gpkgfile, layer=f'gleba-dissolv-{estado}')

## Criando os arquivos de floresta pública

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    floresta_B_uf = floresta[floresta['tipo']=='TIPO B'].overlay(uf_select, how='intersection')
    floresta_dissolv = floresta_B_uf.dissolve()
    floresta_B_uf.to_file(gpkgfile, layer=f'floresta-tipo-b-{estado}')
    floresta_dissolv.to_file(gpkgfile, layer=f'floresta-tipo-b-dissolv-{estado}')

## Criando arquivos de unidades de conservação

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    uc_uf = uc.overlay(uf_select, how='intersection')
    uc_dissolv = uc_uf.dissolve()
    uc_uf.to_file(gpkgfile, layer=f'uc-{estado}')
    uc_dissolv.to_file(gpkgfile, layer=f'uc-dissolv-{estado}')

## Criando terra indigena

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    ti_uf = ti.overlay(uf_select, how='intersection')
    ti_dissolv = ti_uf.dissolve()
    ti_uf.to_file(gpkgfile, layer=f'ti-{estado}')
    ti_dissolv.to_file(gpkgfile, layer=f'ti-dissolv-{estado}')

## Assentamento

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    pa_uf = pa.overlay(uf_select, how='intersection')
    pa_dissolv = pa_uf.dissolve()
    pa_uf.to_file(gpkgfile, layer=f'pa-{estado}')
    pa_dissolv.to_file(gpkgfile, layer=f'pa-dissolv-{estado}')

## Quilombola

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    tq_uf = tq.overlay(uf_select, how='intersection')
    if tq_uf.shape[0] >0:
        tq_dissolv = tq_uf.dissolve()
        tq_uf.to_file(gpkgfile, layer=f'tq-{estado}')
        tq_dissolv.to_file(gpkgfile, layer=f'tq-dissolv-{estado}')

# SIGEF

In [ ]:
estados = ['Acre', 'Amazonas', 'Rondônia', 'Amapá', 'Tocantins', 
           'Roraima', 'Pará', 'Maranhão', 'Mato Grosso']
for estado in estados:
    uf_select = uf[uf['nm_uf']==estado]
    sigef_uf = sigef.overlay(uf_select, how='intersection')
    sigef_dissolv = sigef_uf.dissolve()
    sigef_uf.to_file(gpkgfile, layer=f'sigef-{estado}')
    sigef_dissolv.to_file(gpkgfile, layer=f'sigef-dissolv-{estado}')